LSTM trained on gridded forcings for each station

In [1]:
%load_ext autoreload
%autoreload 2
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from sklearn import preprocessing
import netCDF4 as nc
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from src import load_data, evaluate
import torch.autograd as autograd

In [2]:
USE_CUDA = False
if torch.cuda.is_available():
    print('CUDA Available')
    USE_CUDA = True
device = torch.device('cuda' if USE_CUDA else 'cpu')

writer = SummaryWriter()

CUDA Available


In [3]:
data_runoff = load_data.load_discharge_gr4j_vic()

/home/mgauch/runoff-nn/src/load_data.py:16: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(os.path.join(dir, f), skiprows=2, skipfooter=1, index_col=False, header=None, names=['runoff'], na_values='-1.2345')
/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [4]:
# For each station, read which grid cells belong to its subwatershed
station_cell_mapping = pd.read_csv('station_cell_mapping.csv', skiprows=1, names=['station', 'lat', 'lon', 'row', 'col', 'area'])

In [5]:
rdrs_data = load_data.load_rdrs_forcings()

/home/mgauch/runoff-nn/src/load_data.py:35: RuntimeWarning: invalid value encountered in greater
  var_data = pd.DataFrame(rdrs_nc[var][:].reshape(43825,34*39))


In [6]:
class LSTMRegression(nn.Module):
        def __init__(self, input_dim, hidden_dim, num_layers, batch_size):
            super(LSTMRegression, self).__init__()
            self.batch_size = batch_size
            self.hidden_dim = hidden_dim
            self.num_layers = num_layers
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers)
            self.linear = nn.Linear(hidden_dim, 1)
            self.hidden = self.init_hidden()
        def init_hidden(self):
            return (torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device),
                    torch.randn(self.num_layers, self.batch_size, self.hidden_dim, device=device))

        def forward(self, input):
            lstm_out, self.hidden = self.lstm(input, self.hidden)
            return self.linear(lstm_out[-1])

In [7]:
predictions = {}
actuals = {}
seq_len = 7 * 24
train_start = datetime.strptime('2010-01-01', '%Y-%m-%d') + timedelta(days=seq_len // 24 + 1)
train_end = '2013-12-31'
test_start = '2014-01-01'
test_end = '2014-12-31'

for station in data_runoff['station'].unique():
    print(station)
    station_runoff = data_runoff[data_runoff['station'] == station].set_index('date')
    station_cell_ids = 39 * station_cell_mapping[station_cell_mapping['station'] == station]['col'] \
        + station_cell_mapping[station_cell_mapping['station'] == station]['row']
    station_rdrs = rdrs_data.filter(regex='_(' + '|'.join(map(lambda x: str(x), station_cell_ids)) + ')$', axis=1)
    
    if any(station_runoff['runoff'].isna()):
        print('Station', station, 'had NA runoff values. Skipping.')
        continue
    
    station_train = station_rdrs.loc[train_start : train_end]
    station_test = station_rdrs.loc[test_start : test_end]
    num_train_days = len(pd.date_range(train_start, train_end, freq='D'))
    
    x = np.zeros((seq_len, len(pd.date_range(train_start, test_end, freq='D')), station_rdrs.shape[1]))
    for day in range(x.shape[1]):
        x[:,day,:] = station_rdrs[train_start - timedelta(hours = seq_len - 1) + timedelta(days=day) : train_start + timedelta(days=day)]
    
    # Scale training data
    scalers = []  # save scalers to apply them to test data later
    x_train = x[:,:num_train_days,:]
    for i in range(x.shape[2]):
        scalers.append(preprocessing.StandardScaler())
        x_train[:,:,i] = scalers[i].fit_transform(x_train[:,:,i].reshape((-1, 1))).reshape(x_train[:,:,i].shape)
    x_train = torch.from_numpy(x_train).float().to(device)
    y_train = torch.from_numpy(station_runoff.loc[train_start:train_end, 'runoff'].to_numpy()).float().to(device)
    
    # Prepare model
    H = 100
    batch_size = 3
    lstm_layers = 3
    model = LSTMRegression(station_rdrs.shape[1], H, lstm_layers, batch_size).to(device)
    loss_fn = torch.nn.MSELoss(reduction='mean')
    
    # Train model
    learning_rate = 3e-3
    patience = 30
    min_improvement = 0.1
    best_loss_model = (-1, np.inf, None)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(300):
        epoch_losses = []
        for i in range(num_train_days // batch_size):
            model.hidden = model.init_hidden()
            y_pred = model(x_train[:,i*batch_size : (i+1)*batch_size,:])

            loss = loss_fn(y_pred, y_train[i*batch_size : (i+1)*batch_size].reshape((batch_size,1))).to(device)
            epoch_losses.append(loss.item())
            if i % 50 == 0:
                print(epoch, i, loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        epoch_loss = np.array(epoch_losses).mean()
        print('Epoch', epoch, 'mean loss:', epoch_loss)
        writer.add_scalar('loss_' + station, epoch_loss, epoch)
        if epoch_loss < best_loss_model[1] - min_improvement:
            best_loss_model = (epoch, epoch_loss, model.state_dict())  # new best model
        elif epoch > best_loss_model[0] + patience:
            print('Patience exhausted in epoch {}. Best loss was {}'.format(epoch, best_loss_model[1]))
            break

    model.load_state_dict(best_loss_model[2])
    model.eval()        
    
    # scale test data
    x_test = x[:,num_train_days:,:]
    for i in range(x.shape[2]):
        x_test[:,:,i] = scalers[i].transform(x_test[:,:,i].reshape((-1, 1))).reshape(x_test[:,:,i].shape)
    # if batch size doesn't align with number of samples, add dummies to the last batch
    if x_test.shape[1] % batch_size != 0:
        x_test = np.concatenate([x_test, np.zeros((x_test.shape[0], batch_size - (x_test.shape[1] % batch_size), x_test.shape[2]))], axis=1)
    
    x_test = torch.from_numpy(x_test).float().to(device)
    predict = station_runoff[test_start:test_end].copy()
    predict['runoff'] = np.nan
    pred_array = np.array([])
    print('  Predicting')
    for i in range(x_test.shape[1] // batch_size):
        pred_array = np.concatenate([pred_array, model(x_test[:,i*batch_size : (i+1)*batch_size,:]).detach().cpu().numpy().reshape(batch_size)])
    predict['runoff'] = pred_array[:predict.shape[0]]  # ignore dummies
    predictions[station] = predict
    actuals[station] = station_runoff['runoff'].loc[test_start:test_end]

04159900
0 0 0.33438435196876526
0 50 4.357104778289795
0 100 0.11054138094186783
0 150 206.10227966308594
0 200 27.45200538635254
0 250 10.153582572937012
0 300 7.070493221282959
0 350 0.3141299784183502
0 400 172.7470703125
0 450 12.908111572265625
Epoch 0 mean loss: 40.991088499558856
1 0 2.071673631668091
1 50 5.728527069091797
1 100 1.523364543914795
1 150 222.27684020996094
1 200 24.09910774230957
1 250 8.655527114868164
1 300 7.966115474700928
1 350 0.8803133964538574
1 400 178.46185302734375
1 450 12.32868480682373
Epoch 1 mean loss: 41.42946406429988
2 0 2.164013624191284
2 50 5.873443126678467
2 100 1.6537723541259766
2 150 222.8270721435547
2 200 23.4737548828125
2 250 8.468053817749023
2 300 8.169610023498535
2 350 1.0932122468948364
2 400 180.13409423828125
2 450 12.136271476745605
Epoch 2 mean loss: 41.401151459823815
3 0 2.3047993183135986
3 50 5.962428569793701
3 100 1.7579154968261719
3 150 223.30126953125
3 200 22.847333908081055
3 250 8.536177635192871
3 300 8.169886

In [8]:
nse_list = []
for station, predict in predictions.items():
    nse = evaluate.evaluate_daily(station, predict['runoff'], actuals[station], writer=writer)
    nse_list.append(nse)
    
    print(station, '\tNSE: (clipped to 0)', nse_list[-1])

print('Median NSE (clipped to 0)', np.median(nse_list), '/ Min', np.min(nse_list), '/ Max', np.max(nse_list))

/home/mgauch/runoff-nn/gwf/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


04159900 	NSE: (clipped to 0) -0.13952368532069737
02GC010 	NSE: (clipped to 0) 0.21747001359216434
04215500 	NSE: (clipped to 0) 0.15556095414785454
04174500 	NSE: (clipped to 0) -0.10125588949128606
04165500 	NSE: (clipped to 0) 0.2177243570094769
02GB001 	NSE: (clipped to 0) 0.1550401017670241
04200500 	NSE: (clipped to 0) 0.3943985384482531
04199500 	NSE: (clipped to 0) 0.2977971791336741
04177000 	NSE: (clipped to 0) -0.15105400954692483
04208504 	NSE: (clipped to 0) 0.4255942798627115
04207200 	NSE: (clipped to 0) -0.03831456595784588
04213000 	NSE: (clipped to 0) 0.115781607070646
02GE007 	NSE: (clipped to 0) 0.08658264441600783
02GG009 	NSE: (clipped to 0) -0.042054732391924254
04195820 	NSE: (clipped to 0) -0.39561915745681375
04159492 	NSE: (clipped to 0) -0.06253121434296016
04161820 	NSE: (clipped to 0) 0.21120847406076282
02GG003 	NSE: (clipped to 0) 0.28270496824853386
04196800 	NSE: (clipped to 0) -0.22162756772531278
04215000 	NSE: (clipped to 0) 0.16536627099947376
041

In [9]:
writer.close()